In [2]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install -U \
langchain-community==0.3.5 \
pymupdf==1.24.13 \
faiss-gpu==1.7.2 \
SQLAlchemy==2.0.35 \
dataclasses-json==0.6.7 \
httpx-sse==0.4.0 \
pydantic-settings==2.6.1 \
marshmallow==3.23.1 \
python-dotenv==1.0.1 \
typing-inspect==0.9.0 \
mypy-extensions==1.0.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 20.4 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.2.10
    Uninstalling langsmith-0.2.10:
      Successfully uninstalled langsmith-0.2.10


In [5]:
import torch
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# PDF 로드 및 텍스트 분할
loader = PyMuPDFLoader("/content/drive/MyDrive/Colab Notebooks/pdf/BBS_202402151054353090.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
text_contents = [doc.page_content for doc in texts]

# 임베딩 모델 설정
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# 벡터 스토어 설정
vectorstore = FAISS.from_texts(text_contents, embeddings)

# Qwen/Qwen2.5-1.5B-Instruct 모델 설정
model_name = "Qwen/Qwen2.5-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")
llm = HuggingFacePipeline(
    pipeline=pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.3,
        top_p=0.85,
        repetition_penalty=1.2,
        device=0 if torch.cuda.is_available() else -1
    )
)

# Query expansion 함수
def expand_query(original_query):
    expansion_prompt = f"다음 질문을 확장하여 관련된 다양한 키워드와 문구를 간결하게 한국어로 생성해주세요. 설명이나 문장 형태의 답변은 하지 마세요. 원래 질문: '{original_query}'"
    expanded_query = llm(expansion_prompt).split(original_query)[-1]
    return expanded_query

# 원래 쿼리
original_query = "국내 고령화 전망에 대해 알려주세요"

# 쿼리 확장
expanded_query = expand_query(original_query)
print(f"확장된 쿼리: {expanded_query}")

# 확장된 쿼리로 검색 수행
search_results = vectorstore.similarity_search(expanded_query, k=5)

# 결과 출력
for i, doc in enumerate(search_results):
    print(f"관련 문서 {i+1}:")
    print(doc.page_content[:100] + "...\n")

<ipython-input-5-93170116b7cb>:17: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Device set to use cpu
<ipython-input-5-93170116b7cb>:26: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(
<ipython-input-5-93170116b7cb>:43: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  expanded_query = llm(expansion_prompt).split(original_query)[-1]


확장된 쿼리: ' 

1) 국내 고령화는 어떤 특징이 있나요?
2) 고령사회가 가져올 영향과 문제점有哪些？
3) 어떻게 해야 고령화 사회에서 더 나아질 수 있을까요? 
4) 고령화 시대에는 무엇들이 중요할까?

5) 고령화 현상으로 인한 경제적 변화와 그 해결책들은 뭐인가요?
6) 고령화 대응 방안 중에서 가장 중요한 것은 무엇일까요?
7) 고령화 사회에서는 건강 관리 및养老服务에 대한 관심이 커졌습니다, 이에 대한 이유는 무엇입니까?
8) 고령화 사회에 적합한 교육 방법론은 무엇인지 궁금합니다.
9) 고령화 사회에서 자원봉사 활동 참여율 증가는 이유는 무엇인가요?
10) 고령화 사회에 맞게 준비해야 할 사항들 중 하나는 무엇인가요?


1) 국내 고령화는 주로 60세 이상 노년층의 비중이 늘고 있으며, 특히 70세 이상 노인이 급격히 증가하고 있습니다. 또한, 가족 구성도 변형되며, 일자리를 잃거나 불평등한 근무 환경 등으로 인해 직업 생활에도 어려움을 겪고 있습니다.

2) 고령사회는 노인 문제가 심각하게 발생하며, 복지 서비스 부족, 의료비 상승, 장애인 지원 부재 등의 문제점을 초래합니다. 또한, 경제적인 위기와 육체적 피로감, 정신적 스트레스 등을 경험하는 경우도 많습니다.

3) 고령화 사회에서 더 나아질 수 있는 방법은 다음과 같습니다:
   - 기존 보건 체계 개선
   - 재정 조달과 예산 효율성 제고
   - 지역별 맞춤형 병행 서비스 제공
   - 여성·장애인·노약인 등弱势群体의 권리 보호 강화

4) 고령화 시대에는 공공기관, 학교, 종교 단위 등 여러 분야에서 중요성이 높아집니다. 예를 들어, 공공부문에서는 노인맞춤서비스 도입, 지속 가능한 발전 계획 구현; 학교에서는 성장 과학 캠프 운영, 취미활동 프로그램 개발; 종교 단위에서는 신앙생활 유지 위한 연수 및 교육 제공 등입니다.

5) 고령화 현상을 통해 나타난 경제적 변화는 노인 세대의 소득 감소와 저출산-연령증대 패턴, 그리고 노후 생활비용 증가 등이 있습니다. 이를 해결하기 위해